# CNN scraping
This notebook aims to build a scaper of the news in https://www.cnnbrasil.com.br/

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

In [2]:
driver.get('https://www.cnnbrasil.com.br/politica/')

In [9]:
scrolls = 10

current_height = driver.execute_script("return document.body.scrollHeight")

for i in range(scrolls):
    # scroll to the end of the page
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
    sleep(1)
    
    # handle the 'Aceitar Cokkies' popup
    # cookies_button = driver.find_elements(By.CSS_SELECTOR, value='#onetrust-reject-all-handler')
    # if cookies_button:
    #     cookies_button[0].click()
    #     sleep(1)
    
    try:
        driver.find_element(By.CSS_SELECTOR, value='.block-list-get-more-btn').click()
    except Exception:
        print('Error clicking')
    
    
    
    # repeat the scroll
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight - 1000)")
    current_height = driver.execute_script("return document.body.scrollHeight")
        
    print(f'Scroll {i + 1}, height={current_height}')
    
    # time for the page to load
    sleep(4)

Error clicking
Scroll 1, height=22780


KeyboardInterrupt: 

In [4]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'lxml')

In [5]:
content_block = soup.find('div', class_ = 'col__l--9 col--12')

In [6]:
news_list = content_block.find_all('li', class_ = 'home__list__item')
len(news_list)

70

In [7]:
titles = []
times = []
themes = []

for news in news_list:
    title = news.find('h3', class_ = 'news-item-header__title')
    titles.append(title.text)

    time = news.find('span', 'home__title__date')
    times.append(None if time is None else time.text)
    
    themes.append('Política')
    
data = pd.DataFrame({
    'Title': titles,
    'Time': times,
    'Theme': themes
})

data

,Title,Time,Theme
0,PGR rejeita pedido para incluir “gabinete da ...,11/07/2024 às 21:35,Política
1,"Após pedido de Xuxa, Haddad e Janja divulgam ...",11/07/2024 às 21:20,Política
2,Mulher de Eduardo Bolsonaro o compara a canto...,11/07/2024 às 21:08,Política
3,Abin paralela: Servidores de carreira defende...,11/07/2024 às 21:02,Política
4,Mais de 30 entidades assinam nota contra PEC ...,11/07/2024 às 20:56,Política
...,...,...,...
65,\n\t\t\t\t\t\tIndústria pode ser proibida de d...,\n\t\t\t\t\t\t11/07/2024 às 04:11\t\t\t\t\t,Política
66,"\n\t\t\t\t\t\t“Carne não é a vilã”, diz à CNN ...",\n\t\t\t\t\t\t11/07/2024 às 00:04\t\t\t\t\t,Política
67,"\n\t\t\t\t\t\t“Se não é a ideal, é a possível”...",\n\t\t\t\t\t\t10/07/2024 às 23:58\t\t\t\t\t,Política
68,\n\t\t\t\t\t\tIsenção para carne: saiba como v...,\n\t\t\t\t\t\t10/07/2024 às 23:35\t\t\t\t\t,Política


In [8]:
def convert_to_datetime(time_str):
    # Remove the 'às' and strip any extra whitespace
    cleaned_str = time_str.strip().replace(' às ', ' ')
    # Convert to datetime using the appropriate format
    return datetime.strptime(cleaned_str, '%d/%m/%Y %H:%M')

# Apply the function to the series
data['Time'] = data['Time'].apply(convert_to_datetime)

data['Title'] = data['Title'].str.strip()

data

,Title,Time,Theme
0,PGR rejeita pedido para incluir “gabinete da o...,2024-07-11 21:35:00,Política
1,"Após pedido de Xuxa, Haddad e Janja divulgam v...",2024-07-11 21:20:00,Política
2,Mulher de Eduardo Bolsonaro o compara a cantor...,2024-07-11 21:08:00,Política
3,Abin paralela: Servidores de carreira defendem...,2024-07-11 21:02:00,Política
4,Mais de 30 entidades assinam nota contra PEC d...,2024-07-11 20:56:00,Política
...,...,...,...
65,Indústria pode ser proibida de descartar filho...,2024-07-11 04:11:00,Política
66,"“Carne não é a vilã”, diz à CNN deputado que r...",2024-07-11 00:04:00,Política
67,"“Se não é a ideal, é a possível”, diz Lira apó...",2024-07-10 23:58:00,Política
68,Isenção para carne: saiba como votaram os depu...,2024-07-10 23:35:00,Política
